In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scripts.utils import create_data

from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [31]:
# ==========================
# IMPORT DATA
# ==========================
X, y = create_data()
random_state = 42

# subsample for heavy compute methods
size = 5000
sub_idx = np.random.choice(len(X), size=size, replace=False)
X_sub, y_sub = X.iloc[sub_idx], y.iloc[sub_idx]

(70000, 784)
(70000,)


Kernel PCA.

In [34]:
# Fit Kernel PCA
r = 4
gamma = 0.01

kernel_pca = KernelPCA(
    n_components=r,
    kernel='rbf',
    gamma=gamma,
    random_state=random_state
)

# Kernel PCA is computationally expensive because it constructs an n×n kernel matrix, which scales quadratically with the number of samples. 
# To make the method tractable for the 70,000 MNIST observations, I randomly subsampled 5,000 points, which retains the manifold structure while allowing efficient computation. 

X_kpca = kernel_pca.fit_transform(X=X_sub) 
kpca_df = pd.DataFrame(X_kpca[:, :r], columns=[f'KPCA{i+1}' for i in range(r)])  
kpca_df['digit_name'] = y_sub.values

In [35]:
kpca_df

,KPCA1,KPCA2,KPCA3,KPCA4,digit_name
0,-0.002315,-0.000896,-0.102221,-0.002512,3
1,0.010717,-0.016860,0.012774,0.012722,0
2,0.001993,-0.004143,0.000338,0.015497,1
3,0.000133,0.007918,-0.018003,0.003649,4
4,-0.053440,-0.006007,0.075370,0.049569,1
...,...,...,...,...,...
4995,0.001941,-0.008666,0.000030,0.006411,4
4996,-0.002360,0.009069,0.000346,-0.005141,4
4997,-0.001097,0.004680,0.000174,-0.001936,3
4998,0.001035,-0.003890,0.000050,0.004955,7


In [ ]:
# Center Data 
X_centered = StandardScaler(with_std=False).fit_transform(X_sub)
X_tr, X_ts, y_tr, y_ts = train_test_split(X_centered, y_sub, test_size=0.2, random_state=random_state, stratify=y_sub)

# --- Hyperparameter grid ---
r_list = [10, 20, 40, 60, 80]
gamma_list = [1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 1e-1, 1]

results = []

for r in r_list:
    for gamma in gamma_list:
        kpca = KernelPCA(n_components=r, kernel='rbf', gamma=gamma, random_state=random_state, n_jobs=-1)
        Z_tr = kpca.fit_transform(X_tr)
        Z_ts = kpca.transform(X_ts)

        clf = LogisticRegression(max_iter=1000, multi_class='multinomial', n_jobs=-1)
        clf.fit(Z_tr, y_tr)
        acc = accuracy_score(y_ts, clf.predict(Z_ts))

        results.append((r, gamma, acc))
        print(f"r={r:>3}, gamma={gamma:.1e}, Accuracy={acc:.4f}")

# Find best
best = max(results, key=lambda t: t[2])
print(f"\nBest: r={best[0]}, gamma={best[1]:.1e}, Accuracy={best[2]:.4f}")

In [ ]:
# --- Heatmap visualization ---
r_vals = sorted(set(r_list))
gamma_vals = sorted(set(gamma_list))
acc_matrix = np.zeros((len(r_vals), len(gamma_vals)))

for r, g, acc in results:
    i = r_vals.index(r)
    j = gamma_vals.index(g)
    acc_matrix[i, j] = acc

plt.figure(figsize=(6,4))
plt.imshow(acc_matrix, cmap='viridis', aspect='auto')
plt.colorbar(label="Test Accuracy")
plt.xticks(range(len(gamma_vals)), [f"{g:.0e}" for g in gamma_vals])
plt.yticks(range(len(r_vals)), r_vals)
plt.xlabel("Gamma")
plt.ylabel("Number of Components (r)")
plt.title("Kernel PCA - Hyperparameter Tuning via Downstream Accuracy")
plt.tight_layout()
# plt.savefig("docs/kernelpca_hyperparam_heatmap.png", dpi=150)
plt.show()